In [ ]:
# # Cài đặt các gói cần thiết
# !pip install pytorch-lightning
# !pip install torchmetrics
# !pip install transformers
# !pip install datasets

In [ ]:
# Import các thư viện cần thiết
import os
import zipfile
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
from urllib.request import urlretrieve

import pandas as pd
from tqdm import tqdm

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchmetrics
from datasets import load_dataset
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          DataCollatorWithPadding)

# Đặt seed để đảm bảo tính tái tạo


In [ ]:
class TqdmUpTo(tqdm):
    """From https://github.com/tqdm/tqdm/blob/master/examples/tqdm_wget.py"""

    def update_to(self, blocks=1, bsize=1, tsize=None):
        """
        Parameters
        ----------
        blocks: int, optional
            Number of blocks transferred so far [default: 1].
        bsize: int, optional
            Size of each block (in tqdm units) [default: 1].
        tsize: int, optional
            Total size (in tqdm units). If [default: None] remains unchanged.
        """
        if tsize is not None:
            self.total = tsize  # pylint: disable=attribute-defined-outside-init
        self.update(blocks * bsize - self.n)  # will also set self.n = b * bsize


def download_url(url, filename, directory='.'):
    """Download a file from url to filename, with a progress bar."""
    if not os.path.exists(directory):
        os.makedirs(directory)
    path = os.path.join(directory, filename)

    with TqdmUpTo(unit="B", unit_scale=True, unit_divisor=1024, miniters=1) as t:
        urlretrieve(url, path, reporthook=t.update_to, data=None)  # nosec
    return path

def _load_data_from(data_dir: Union[str, Path]):
    """Load dữ liệu cảm xúc từ tệp văn bản"""
    fnames = ['sentiments.txt', 'sents.txt', 'topics.txt']
    sentiments = []
    sents = []
    topics = []
    for name in fnames:
        with open(f"{data_dir}/{name}", 'r') as f:
            if name == "sentiments.txt":
                sentiments = [int(line.strip()) for line in f.readlines()]
            elif name == "sents.txt":
                sents = [line.strip() for line in f.readlines()]        
            else:
                topics = [int(line.strip()) for line in f.readlines()]
    return sents, sentiments, topics

def _save_to_csv(file_path: Union[str, Path], data):
    """Chuyển đổi dữ liệu sang định dạng CSV"""
    sents, sentiments, topics = data
    df = pd.DataFrame({
        "sents": sents,
        "labels": sentiments,
        "topics": topics
    })
    df.to_csv(file_path, index=False)
    return file_path

In [ ]:
class UIT_VSFC(pl.LightningDataModule):
    """
    UIT-VSFC: Vietnamese Students' Feedback Corpus for sentiment analysis
    """
    def __init__(self, tokenizer, opts: Dict[str, Any]):
        super().__init__()
        self.tokenizer = tokenizer
        self.batch_size = opts['batch_size']
        self.num_workers = opts['num_workers']
        self.on_gpu = opts['on_gpu']
        self.data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        self.dataset = None
        self.mapping = {"negative": 0, "neutral": 1, "positive": 2}
        self.inverse_mapping = {v: k for k, v in self.mapping.items()}
        
        # Tải dataset luôn khi khởi tạo để tránh lỗi setup
        self.raw_datasets = load_dataset("uitnlp/vietnamese_students_feedback")
        
    def prepare_data(self):
        # Phương thức này chỉ gọi một lần và không cần trả về gì
        pass

    def setup(self, stage=None):
        # Kiểm tra cấu trúc dữ liệu
        sample = self.raw_datasets['train'][0]
        print("Cấu trúc dữ liệu mẫu:", sample)
        
        # Hàm tokenize đầu vào
        def tokenize_function(examples):
            # Thêm truncation=True để đảm bảo độ dài phù hợp
            return self.tokenizer(examples['sentence'], truncation=True, max_length=256)
        
        if self.dataset is None:
            # Tạo dataset nếu chưa được tạo
            splits = {
                'train': self.raw_datasets['train'],
                'dev': self.raw_datasets['validation'],
                'test': self.raw_datasets['test']
            }
            
            # Áp dụng tokenize và định dạng
            self.dataset = {}
            for split_name, split_data in splits.items():
                print(f"Xử lý split: {split_name}")
                # Áp dụng tokenizer
                try:
                    # Áp dụng tokenizer
                    tokenized_data = split_data.map(
                        tokenize_function,
                        batched=True,
                        remove_columns=['sentence', 'topic']  # Xóa cột không cần thiết
                    )
                    
                    # In thông tin cột để debug
                    print(f"Các cột sau khi tokenize: {tokenized_data.column_names}")
                    
                    # Lưu lại dữ liệu tokenized trước khi chuyển đổi định dạng
                    self.dataset[split_name] = tokenized_data
                    
                    # Tạo dataset với các trường cần thiết
                    formatted_data = tokenized_data.with_format(
                        type='torch',
                        columns=['input_ids', 'attention_mask', 'sentiment'],
                        output_all_columns=False  # Chỉ giữ lại các cột đã chỉ định
                    )
                    
                    # Kiểm tra dữ liệu đã được định dạng
                    if formatted_data is not None:
                        # Đổi tên cột
                        formatted_data = formatted_data.rename_column('sentiment', 'labels')
                        self.dataset[split_name] = formatted_data
                    else:
                        print(f"Lỗi: Không thể định dạng dữ liệu cho split {split_name}")
                
                except Exception as e:
                    print(f"Lỗi khi xử lý split {split_name}: {str(e)}")
                    # Sử dụng cách tiếp cận thủ công nếu cách trên không hoạt động
                    try:
                        print("Thử phương pháp thay thế...")
                        # Tiếp cận thủ công để tạo tensor
                        tokenized_data = split_data.map(
                            tokenize_function,
                            batched=True
                        )
                        
                        # Chuyển đổi thành dict of lists
                        data_dict = {
                            'input_ids': tokenized_data['input_ids'],
                            'attention_mask': tokenized_data['attention_mask'],
                            'labels': tokenized_data['sentiment']  # Đổi tên ngay tại đây
                        }
                        
                        # Tạo datasets từ dict
                        from datasets import Dataset
                        self.dataset[split_name] = Dataset.from_dict(data_dict).with_format("torch")
                    except Exception as e2:
                        print(f"Cả hai phương pháp đều thất bại cho split {split_name}: {str(e2)}")
                        continue
    
    def train_dataloader(self):
        # Đảm bảo setup đã được gọi
        if self.dataset is None:
            self.setup()
            
        if 'train' not in self.dataset or self.dataset['train'] is None:
            raise ValueError("Train dataset không khả dụng")
            
        return DataLoader(
            self.dataset['train'],
            shuffle=True,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=self.on_gpu,
            collate_fn=self.data_collator
        )
    
    def val_dataloader(self):
        # Đảm bảo setup đã được gọi
        if self.dataset is None:
            self.setup()
            
        if 'dev' not in self.dataset or self.dataset['dev'] is None:
            raise ValueError("Validation dataset không khả dụng")
            
        return DataLoader(
            self.dataset['dev'],
            shuffle=False,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=self.on_gpu,
            collate_fn=self.data_collator
        )
    
    def test_dataloader(self):
        # Đảm bảo setup đã được gọi
        if self.dataset is None:
            self.setup()
            
        if 'test' not in self.dataset or self.dataset['test'] is None:
            raise ValueError("Test dataset không khả dụng")
            
        return DataLoader(
            self.dataset['test'],
            shuffle=False,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=self.on_gpu,
            collate_fn=self.data_collator
        )

In [ ]:
class PhoBERT(pl.LightningModule):
    def __init__(self, lr, weight_decay):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)
        self.lr = lr
        self.weight_decay = weight_decay

        # Định nghĩa các metrics
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3)
        self.val_f1 = torchmetrics.F1Score(task="multiclass", num_classes=3)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3)
        self.test_f1 = torchmetrics.F1Score(task="multiclass", num_classes=3)
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
    
    def training_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        loss, logits = outputs.loss, outputs.logits
        sentiments = batch['labels']
        scores = F.softmax(logits, dim=-1)
        self.train_acc(scores, sentiments)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        loss, logits = outputs.loss, outputs.logits
        sentiments = batch['labels']
        scores = F.softmax(logits, dim=-1)
        self.val_acc(scores, sentiments)
        self.val_f1(scores, sentiments)
        
        # Cải thiện cách log metrics
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_f1', self.val_f1, on_step=False, on_epoch=True, prog_bar=True, logger=True, sync_dist=True)

    def test_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        logits = outputs.logits
        sentiments = batch['labels']
        scores = F.softmax(logits, dim=-1)
        self.test_acc(scores, sentiments)
        self.test_f1(scores, sentiments)
        
        # Cải thiện cách log metrics
        self.log('test_acc', self.test_acc, on_step=False, on_epoch=True, logger=True)
        self.log('test_f1', self.test_f1, on_step=False, on_epoch=True, logger=True)

    # Thêm phương thức on_validation_epoch_end để đảm bảo metrics được tính toán đầy đủ
    def on_validation_epoch_end(self):
        # Log lại metrics một lần nữa ở cuối epoch
        self.log('val_acc_epoch', self.val_acc.compute(), prog_bar=True, logger=True)
        self.log('val_f1_epoch', self.val_f1.compute(), prog_bar=True, logger=True)
        
        # In thông tin để debug
        print(f"Epoch end val_f1: {self.val_f1.compute():.4f}, val_acc: {self.val_acc.compute():.4f}")

In [ ]:
    def validation_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        loss, logits = outputs.loss, outputs.logits
        sentiments = batch['labels']
        scores = F.softmax(logits, dim=-1)
        self.val_acc(scores, sentiments)
        self.val_f1(scores, sentiments)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_f1', self.val_f1, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        logits = outputs.logits
        sentiments = batch['labels']
        scores = F.softmax(logits, dim=-1)
        self.test_acc(scores, sentiments)
        self.test_f1(scores, sentiments)
        self.log('test_acc', self.test_acc, on_step=False, on_epoch=True, logger=True)
        self.log('test_f1', self.test_f1, on_step=False, on_epoch=True, logger=True)

In [ ]:
# Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Tùy chọn cho datamodule
options = {
    "on_gpu": torch.cuda.is_available(),
    "batch_size": 16,
    "num_workers": 2
}

# Khởi tạo datamodule
datamodule = UIT_VSFC(tokenizer, options)

# Các siêu tham số
lr = 2e-5
weight_decay = 0.01

# Khởi tạo mô hình
model = PhoBERT(lr, weight_decay)

# Thiết lập callback lưu checkpoint
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    dirpath='checkpoints',
    filename='phobert-sentiment-{epoch:02d}-{val_f1:.4f}',
    save_top_k=1,
    mode='max',
)

# Khởi tạo trainer
trainer = pl.Trainer(
    max_epochs=10,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    callbacks=[checkpoint_callback],
    deterministic=True,
)

# Kiểm tra dataloader trước khi huấn luyện (tùy chọn)
datamodule.setup()
batch = next(iter(datamodule.train_dataloader()))
print("Batch keys:", batch.keys())
print("Input ids shape:", batch['input_ids'].shape)
print("Labels shape:", batch['labels'].shape if 'labels' in batch else "Labels not found")

# Huấn luyện mô hình
trainer.fit(model, datamodule)

In [ ]:
# Kiểm tra mô hình sử dụng checkpoint tốt nhất
test_results = trainer.test(ckpt_path=checkpoint_callback.best_model_path, datamodule=datamodule)
print(f"Kết quả kiểm tra: {test_results}")

In [ ]:
# Hiển thị kết quả và so sánh với baseline
import matplotlib.pyplot as plt
import numpy as np

# Kết quả từ mô hình của chúng ta
our_results = {
    'Accuracy': test_results[0]['test_acc'],
    'F1 Score': test_results[0]['test_f1']
}

# Kết quả từ bài báo
paper_results = {
    'Accuracy': 0.879,
    'F1 Score': 0.879
}

# Tạo biểu đồ cột để so sánh kết quả
models = ['PhoBERT (Cài đặt của chúng ta)', 'MaxEnt (Bài báo gốc)']
metrics = ['Accuracy', 'F1 Score']

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, [our_results[m] for m in metrics], width, label=models[0])
rects2 = ax.bar(x + width/2, [paper_results[m] for m in metrics], width, label=models[1])

ax.set_ylabel('Điểm số')
ax.set_title('So sánh hiệu suất trên tập dữ liệu UIT-VSFC')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

ax.bar_label(rects1, padding=3, fmt='%.3f')
ax.bar_label(rects2, padding=3, fmt='%.3f')
fig.tight_layout()

plt.show()

In [ ]:
print("Thảo luận:")
print("Mô hình PhoBERT của chúng ta đạt độ chính xác {:.2f}% và điểm F1 {:.2f}%, ".format(
    our_results['Accuracy']*100, our_results['F1 Score']*100))
print("vượt trội hơn baseline MaxEnt từ bài báo gốc {:.2f}% ".format(
    (our_results['Accuracy'] - paper_results['Accuracy'])*100))
print("mà không cần điều chỉnh siêu tham số nhiều.")
print("\nCác cải tiến tiềm năng:")
print("1. Lên lịch cho tốc độ học (learning rate scheduling)")
print("2. Điều chỉnh siêu tham số rộng rãi hơn bằng cách sử dụng Wandb sweeps")
print("3. Thử nghiệm các mô hình pre-trained khác (XLM-RoBERTa, BERTweet, v.v.)")
print("4. Tăng cường dữ liệu cho các lớp không cân bằng")
print("5. Phương pháp tổng hợp kết hợp nhiều mô hình")

In [ ]:
# In ra đường dẫn của checkpoint tốt nhất
print(f"Đường dẫn đến checkpoint tốt nhất: {checkpoint_callback.best_model_path}")

# Nếu bạn muốn sao chép checkpoint tốt nhất vào một thư mục khác
import os
import shutil

best_ckpt = checkpoint_callback.best_model_path
output_dir = '/kaggle/working/checkpoints'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, os.path.basename(best_ckpt))
print(f"Đã sao chép checkpoint tốt nhất vào: {output_file}")